# The answer is 42

**TL;DR:** The solution is at the bottom.

In [527]:
import sympy as sym

## Solving propeller angular velocities

In [528]:
c_bar, p_bar, q_bar, r_bar = sym.symbols('c_bar, p_bar, q_bar, r_bar')
c_bar

c_bar

In [529]:
om_1, om_2, om_3, om_4 = sym.symbols('omega_1, omega_2, omega_3, omega_4')
om_1

omega_1

In [530]:
om_1_sq = om_1 ** 2
om_2_sq = om_2 ** 2
om_3_sq = om_3 ** 2
om_4_sq = om_4 ** 2
om_1_sq

omega_1**2

In [531]:
cpqr = sym.Matrix([c_bar, p_bar, q_bar, r_bar])
cpqr

Matrix([
[c_bar],
[p_bar],
[q_bar],
[r_bar]])

In [532]:
D = sym.Matrix([[1,  1,  1,  1], 
                [1, -1, -1,  1], 
                [1,  1, -1, -1],
                [1, -1,  1, -1]])
D

Matrix([
[1,  1,  1,  1],
[1, -1, -1,  1],
[1,  1, -1, -1],
[1, -1,  1, -1]])

In [533]:
omegas = sym.Matrix([om_1_sq, om_2_sq, om_3_sq, om_4_sq])
omegas

Matrix([
[omega_1**2],
[omega_2**2],
[omega_3**2],
[omega_4**2]])

So we have

In [534]:
sym.Eq(cpqr, D*omegas)

Eq(Matrix([
[c_bar],
[p_bar],
[q_bar],
[r_bar]]), Matrix([
[omega_1**2 + omega_2**2 + omega_3**2 + omega_4**2],
[omega_1**2 - omega_2**2 - omega_3**2 + omega_4**2],
[omega_1**2 + omega_2**2 - omega_3**2 - omega_4**2],
[omega_1**2 - omega_2**2 + omega_3**2 - omega_4**2]]))

Solving the system for $\omega^2_1$, $\omega^2_2$, $\omega^2_3$ and $\omega^2_4$ gives

In [535]:
sym.solve([sym.Eq(cpqr, D*omegas)], [om_1_sq, om_2_sq, om_3_sq, om_4_sq])

{omega_1**2: c_bar/4 + p_bar/4 + q_bar/4 + r_bar/4,
 omega_2**2: c_bar/4 - p_bar/4 + q_bar/4 - r_bar/4,
 omega_3**2: c_bar/4 - p_bar/4 - q_bar/4 + r_bar/4,
 omega_4**2: c_bar/4 + p_bar/4 - q_bar/4 - r_bar/4}

Which is the same as

In [536]:
D.inv()*cpqr

Matrix([
[c_bar/4 + p_bar/4 + q_bar/4 + r_bar/4],
[c_bar/4 - p_bar/4 + q_bar/4 - r_bar/4],
[c_bar/4 - p_bar/4 - q_bar/4 + r_bar/4],
[c_bar/4 + p_bar/4 - q_bar/4 - r_bar/4]])

or `x = A.LUsolve(b)` given $b = Ax$ (so $\Omega = D^{-1} C_{cpqr})$:

In [537]:
D.LUsolve(cpqr)

Matrix([
[c_bar/4 + p_bar/4 + q_bar/4 + r_bar/4],
[c_bar/4 - p_bar/4 + q_bar/4 - r_bar/4],
[c_bar/4 - p_bar/4 - q_bar/4 + r_bar/4],
[c_bar/4 + p_bar/4 - q_bar/4 - r_bar/4]])

## Motor Commands

In [538]:
L, kappa, k_f, k_m = sym.symbols('L, kappa, k_f, k_m')
Ixx, Iyy, Izz = sym.symbols('I_x, I_y, I_z')
F1, F2, F3, F4 = sym.symbols('F_1, F_2, F_3, F_4')
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

Here's the perpendicular distance of the rotors to the center of mass:

In [539]:
l = L / sym.sqrt(2)
l

sqrt(2)*L/2

But for simplicity, let's just go with $l$ for the time being:

In [540]:
l = sym.Symbol('l')
l

l

The collective thrust:

In [541]:
F = F1 + F2 + F3 + F4
F

F_1 + F_2 + F_3 + F_4

Defining $\bar{u}_p$, $\bar{u}_q$ and $\bar{u}_r$:

In [542]:
u_p_bar = tau_x / Ixx
u_q_bar = tau_y / Iyy
u_r_bar = tau_z / Izz
u_p_bar

tau_x/I_x

Defining $\bar{c}$, $\bar{p}$, $\bar{q}$ and $\bar{r}$:

In [543]:
c_bar = F
p_bar = Ixx * u_p_bar / (k_f * l)
q_bar = Iyy * u_q_bar / (k_f * l)
r_bar = Izz * u_r_bar / k_m

In [544]:
p_bar

tau_x/(k_f*l)

In [545]:
r_bar

tau_z/k_m

We redefine the $C_{cpqr}$ vector using it's actual definitions:

In [546]:
cpqr = sym.Matrix([c_bar, p_bar, q_bar, r_bar])
cpqr

Matrix([
[F_1 + F_2 + F_3 + F_4],
[        tau_x/(k_f*l)],
[        tau_y/(k_f*l)],
[            tau_z/k_m]])

Similar to the equation system in the first section, we have $C_{cpqr} = D \cdot \Omega$:

In [547]:
eq_1 = sym.Eq(cpqr, D*omegas)
eq_1

Eq(Matrix([
[F_1 + F_2 + F_3 + F_4],
[        tau_x/(k_f*l)],
[        tau_y/(k_f*l)],
[            tau_z/k_m]]), Matrix([
[omega_1**2 + omega_2**2 + omega_3**2 + omega_4**2],
[omega_1**2 - omega_2**2 - omega_3**2 + omega_4**2],
[omega_1**2 + omega_2**2 - omega_3**2 - omega_4**2],
[omega_1**2 - omega_2**2 + omega_3**2 - omega_4**2]]))

And solving for $\Omega$ using $\Omega = D^{-1} \cdot C_{cpqr}$ gives

In [548]:
eq_2 = sym.Eq(omegas, D.inv() * cpqr)
eq_2

Eq(Matrix([
[omega_1**2],
[omega_2**2],
[omega_3**2],
[omega_4**2]]), Matrix([
[F_1/4 + F_2/4 + F_3/4 + F_4/4 + tau_z/(4*k_m) + tau_x/(4*k_f*l) + tau_y/(4*k_f*l)],
[F_1/4 + F_2/4 + F_3/4 + F_4/4 - tau_z/(4*k_m) - tau_x/(4*k_f*l) + tau_y/(4*k_f*l)],
[F_1/4 + F_2/4 + F_3/4 + F_4/4 + tau_z/(4*k_m) - tau_x/(4*k_f*l) - tau_y/(4*k_f*l)],
[F_1/4 + F_2/4 + F_3/4 + F_4/4 - tau_z/(4*k_m) + tau_x/(4*k_f*l) - tau_y/(4*k_f*l)]]))

In order to get torque from squared rotational velocities, we need to factor in $k_m$:

In [549]:
taus = sym.Matrix([tau_1, tau_2, tau_3, tau_4])
eq_3 = sym.Eq(taus, k_m * D.inv() * cpqr)
eq_3

Eq(Matrix([
[tau_1],
[tau_2],
[tau_3],
[tau_4]]), Matrix([
[k_m*(F_1 + F_2 + F_3 + F_4)/4 + tau_z/4 + k_m*tau_x/(4*k_f*l) + k_m*tau_y/(4*k_f*l)],
[k_m*(F_1 + F_2 + F_3 + F_4)/4 - tau_z/4 - k_m*tau_x/(4*k_f*l) + k_m*tau_y/(4*k_f*l)],
[k_m*(F_1 + F_2 + F_3 + F_4)/4 + tau_z/4 - k_m*tau_x/(4*k_f*l) - k_m*tau_y/(4*k_f*l)],
[k_m*(F_1 + F_2 + F_3 + F_4)/4 - tau_z/4 + k_m*tau_x/(4*k_f*l) - k_m*tau_y/(4*k_f*l)]]))

And to obtain individual forces from the torques, we need to divide by the perpendicular length:

In [550]:
forces = sym.Matrix([F1, F2, F3, F4])
eq_4 = sym.Eq(forces, k_m / l * D.inv() * cpqr)
eq_4

Eq(Matrix([
[F_1],
[F_2],
[F_3],
[F_4]]), Matrix([
[k_m*(F_1 + F_2 + F_3 + F_4)/(4*l) + tau_z/(4*l) + k_m*tau_x/(4*k_f*l**2) + k_m*tau_y/(4*k_f*l**2)],
[k_m*(F_1 + F_2 + F_3 + F_4)/(4*l) - tau_z/(4*l) - k_m*tau_x/(4*k_f*l**2) + k_m*tau_y/(4*k_f*l**2)],
[k_m*(F_1 + F_2 + F_3 + F_4)/(4*l) + tau_z/(4*l) - k_m*tau_x/(4*k_f*l**2) - k_m*tau_y/(4*k_f*l**2)],
[k_m*(F_1 + F_2 + F_3 + F_4)/(4*l) - tau_z/(4*l) + k_m*tau_x/(4*k_f*l**2) - k_m*tau_y/(4*k_f*l**2)]]))

Note that we can still substitute the sum of the individual forces $F_1$, $F_2$, $F_3$, $F_4$ with the collective thrust $F$:

In [551]:
eq_5 = sym.Eq(forces, k_m / l * D.inv() * cpqr).subs(F, sym.Symbol('F'))
eq_5

Eq(Matrix([
[F_1],
[F_2],
[F_3],
[F_4]]), Matrix([
[F*k_m/(4*l) + tau_z/(4*l) + k_m*tau_x/(4*k_f*l**2) + k_m*tau_y/(4*k_f*l**2)],
[F*k_m/(4*l) - tau_z/(4*l) - k_m*tau_x/(4*k_f*l**2) + k_m*tau_y/(4*k_f*l**2)],
[F*k_m/(4*l) + tau_z/(4*l) - k_m*tau_x/(4*k_f*l**2) - k_m*tau_y/(4*k_f*l**2)],
[F*k_m/(4*l) - tau_z/(4*l) + k_m*tau_x/(4*k_f*l**2) - k_m*tau_y/(4*k_f*l**2)]]))

Note the common factor of $\frac{1}{4\cdot l}$.

Now the problem remains that we do not know the quantities $k_m$ and $k_f$ as far as the code is concerned.

One interesting aspect is in the definitions

$$
\begin{align}
F &= k_f \cdot \omega^2 \\
\tau &= k_m \cdot \omega^2 \\
\tau &= F \cdot l
\end{align}
$$

By substituting $F$, we get

$$
\begin{align}
k_m \cdot \omega^2 &= k_f \cdot \omega^2 \cdot l \\
k_m &= k_f \cdot l
\end{align}
$$

If we substitute occurrences of $k_f \cdot l$ with $k_m$, we arrive at the much shorter

In [552]:
sym.simplify(eq_5.subs(l * kf, km))

Eq(Matrix([
[F_1],
[F_2],
[F_3],
[F_4]]), Matrix([
[(F*k_m + tau_x + tau_y + tau_z)/(4*l)],
[(F*k_m - tau_x + tau_y - tau_z)/(4*l)],
[(F*k_m - tau_x - tau_y + tau_z)/(4*l)],
[(F*k_m + tau_x - tau_y - tau_z)/(4*l)]]))

However, we still have a stray $k_m$ in there. Note that the _values_ of $\tau$ are the same, but the physical meaning may has changed due to normalization we just did. That is, if the math is correct.

## So everything was wrong?

Let's try again.

In [553]:
l, L, kappa, kf, km = sym.symbols('l, L, kappa, k_f, k_m')
Ixx, Iyy, Izz = sym.symbols('I_x, I_y, I_z')
F, F1, F2, F3, F4 = sym.symbols('F, F_1, F_2, F_3, F_4')
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')
tau_1, tau_2, tau_3, tau_4 = sym.symbols('tau_1, tau_2, tau_3, tau_4')

In [554]:
c_bar, p_bar, q_bar, r_bar = sym.symbols('c_bar, p_bar, q_bar, r_bar')
u_p_bar, u_q_bar, u_r_bar = sym.symbols('u_p_bar, u_q_bar, u_r_bar')
om_1, om_2, om_3, om_4 = sym.symbols('omega_1, omega_2, omega_3, omega_4')

Let's set up some relations.

In [555]:
l_eq = sym.Eq(l, L / sym.sqrt(2))
display(l_eq)

Eq(l, sqrt(2)*L/2)

In [556]:
total_thrust = sym.Eq(F, F1 + F2 + F3 + F4)
display(total_thrust)

Eq(F, F_1 + F_2 + F_3 + F_4)

In [557]:
forces = sym.Matrix((F1, F2, F3, F4))
display(forces)

Matrix([
[F_1],
[F_2],
[F_3],
[F_4]])

In [558]:
moments = sym.Matrix((tau_1, tau_2, tau_3, tau_4))
display(moments)

Matrix([
[tau_1],
[tau_2],
[tau_3],
[tau_4]])

In [559]:
omegas = sym.Matrix((om_1, om_2, om_3, om_4))
display(omegas)

Matrix([
[omega_1],
[omega_2],
[omega_3],
[omega_4]])

In [560]:
omegas_sq = sym.Matrix((om_1_sq, om_2_sq, om_3_sq, om_4_sq))
display(omegas_sq)

Matrix([
[omega_1**2],
[omega_2**2],
[omega_3**2],
[omega_4**2]])

### Forces from momentum

In [561]:
F1_from_tau_1 = sym.Eq(F1, kf * om_1_sq)
F2_from_tau_2 = sym.Eq(F2, kf * om_1_sq)
F3_from_tau_3 = sym.Eq(F3, kf * om_1_sq)
F4_from_tau_4 = sym.Eq(F4, kf * om_1_sq)

display(F1_from_tau_1)
display(F2_from_tau_2)
display(F3_from_tau_3)
display(F4_from_tau_4)

Eq(F_1, k_f*omega_1**2)

Eq(F_2, k_f*omega_1**2)

Eq(F_3, k_f*omega_1**2)

Eq(F_4, k_f*omega_1**2)

In [562]:
fs_from_taus = sym.Eq(forces, kf * omegas_sq)
display(fs_from_taus)

Eq(Matrix([
[F_1],
[F_2],
[F_3],
[F_4]]), Matrix([
[k_f*omega_1**2],
[k_f*omega_2**2],
[k_f*omega_3**2],
[k_f*omega_4**2]]))

In other words, the thrust coefficient $k_f$ is thrust per rotational velocity squared.

### Momentum from angular velocities

In [563]:
tau_1_from_om_1_sq = sym.Eq(tau_1, km * om_1_sq)
tau_2_from_om_2_sq = sym.Eq(tau_2, km * om_2_sq)
tau_3_from_om_3_sq = sym.Eq(tau_3, km * om_3_sq)
tau_4_from_om_4_sq = sym.Eq(tau_4, km * om_4_sq)

display(tau_1_from_om_1_sq)
display(tau_2_from_om_2_sq)
display(tau_3_from_om_3_sq)
display(tau_4_from_om_4_sq)

Eq(tau_1, k_m*omega_1**2)

Eq(tau_2, k_m*omega_2**2)

Eq(tau_3, k_m*omega_3**2)

Eq(tau_4, k_m*omega_4**2)

In [564]:
taus_from_omegas = sym.Eq(moments, km * omegas_sq)
display(taus_from_omegas)

Eq(Matrix([
[tau_1],
[tau_2],
[tau_3],
[tau_4]]), Matrix([
[k_m*omega_1**2],
[k_m*omega_2**2],
[k_m*omega_3**2],
[k_m*omega_4**2]]))

In other words, the torque coefficient $k_m$ is torque per rotational velocity squared.

### About kappa

From $k_f$ and $k_m$ we have

$$
\omega^2 = \frac{\tau}{k_m} = \frac{F}{k_f}
$$

and from there,

$$
\frac{\tau}{F} = \frac{k_m}{k_f}
$$

From the code, we have $\kappa$ defined as "torque (Nm) produced by motor per N of thrust produced", so

$$
\begin{align}
\kappa = \frac{\tau}{F} = \frac{k_m}{k_f}
\end{align}
$$

In [565]:
forces_from_torques = sym.Eq(forces, moments / kappa)
torques_from_forces = sym.Eq(moments, forces * kappa)

display(forces_from_torques)
display(torques_from_forces)

Eq(Matrix([
[F_1],
[F_2],
[F_3],
[F_4]]), Matrix([
[tau_1/kappa],
[tau_2/kappa],
[tau_3/kappa],
[tau_4/kappa]]))

Eq(Matrix([
[tau_1],
[tau_2],
[tau_3],
[tau_4]]), Matrix([
[F_1*kappa],
[F_2*kappa],
[F_3*kappa],
[F_4*kappa]]))

### u_bar from momentum

In [566]:
u_p_bar_from_tau_x = sym.Eq(u_p_bar, tau_x / Ixx)
u_q_bar_from_tau_y = sym.Eq(u_q_bar, tau_y / Iyy)
u_r_bar_from_tau_z = sym.Eq(u_r_bar, tau_z / Izz)

display(u_p_bar_from_tau_x)
display(u_q_bar_from_tau_y)
display(u_r_bar_from_tau_z)

Eq(u_p_bar, tau_x/I_x)

Eq(u_q_bar, tau_y/I_y)

Eq(u_r_bar, tau_z/I_z)

### c_bar

In [567]:
c_bar_from_f = sym.Eq(c_bar, F)

display(c_bar_from_f)

Eq(c_bar, F)

### p_bar from u_p_bar

In [568]:
p_bar_from_u_p_bar = sym.Eq(p_bar, Ixx * u_p_bar / (k_f * l))
p_bar_from_tau_x = p_bar_from_u_p_bar.subs(u_p_bar, u_p_bar_from_tau_x.rhs)

display(p_bar_from_u_p_bar)
display(p_bar_from_tau_x)

Eq(p_bar, I_x*u_p_bar/(k_f*l))

Eq(p_bar, tau_x/(k_f*l))

### q_bar from u_q_bar

In [569]:
q_bar_from_u_q_bar = sym.Eq(q_bar, Iyy * u_q_bar / (k_f * l))
q_bar_from_tau_y = q_bar_from_u_q_bar.subs(u_q_bar, u_q_bar_from_tau_y.rhs)

display(q_bar_from_u_q_bar)
display(q_bar_from_tau_y)

Eq(q_bar, I_y*u_q_bar/(k_f*l))

Eq(q_bar, tau_y/(k_f*l))

### r_bar from u_r_bar

In [570]:
r_bar_from_u_r_bar = sym.Eq(r_bar, Izz * u_r_bar / k_m)
r_bar_from_tau_z = r_bar_from_u_r_bar.subs(u_r_bar, u_r_bar_from_tau_z.rhs)

display(r_bar_from_u_r_bar)
display(r_bar_from_tau_z)

Eq(r_bar, I_z*u_r_bar/k_m)

Eq(r_bar, tau_z/k_m)

### Bars are open

In [571]:
cpqr_bars = sym.Eq(sym.Matrix((c_bar, p_bar, q_bar, r_bar)),
                   sym.Matrix((c_bar_from_f.rhs, 
                               p_bar_from_tau_x.rhs, 
                               q_bar_from_tau_y.rhs, 
                               r_bar_from_tau_z.rhs)))
display(cpqr_bars)

Eq(Matrix([
[c_bar],
[p_bar],
[q_bar],
[r_bar]]), Matrix([
[            F],
[tau_x/(k_f*l)],
[tau_y/(k_f*l)],
[    tau_z/k_m]]))

In [572]:
cpqr_bars.subs(F1, F1_from_tau_1.rhs)\
         .subs(F2, F2_from_tau_2.rhs)\
         .subs(F3, F3_from_tau_3.rhs)\
         .subs(F4, F4_from_tau_4.rhs)

Eq(Matrix([
[c_bar],
[p_bar],
[q_bar],
[r_bar]]), Matrix([
[            F],
[tau_x/(k_f*l)],
[tau_y/(k_f*l)],
[    tau_z/k_m]]))

### An attempt at solving it

In [573]:
moments_xyz = sym.Matrix((tau_x, tau_y, tau_z))
display(moments_xyz)

Matrix([
[tau_x],
[tau_y],
[tau_z]])

The roll and pitch moments $\tau_x$ and $\tau_y$ can be determined trivially by the motor forces and the perpendicular distance of the motors; the yaw moment then is the result of the individual torques and determined by their sum. Note, however, that their direction of rotation needs to be incorporated, such that

$$
\tau_z = \tau_1 - \tau_2 + \tau_3 - \tau_4
$$

This gives

In [574]:
eq_tau_xyz = sym.Eq(moments_xyz, sym.Matrix([
                   (F1 - F2 - F3 + F4) * l,
                   (F1 + F2 - F3 - F4) * l,
                   tau_1 - tau_2 + tau_3 - tau_4]))
display(eq_tau_xyz)

Eq(Matrix([
[tau_x],
[tau_y],
[tau_z]]), Matrix([
[    l*(F_1 - F_2 - F_3 + F_4)],
[    l*(F_1 + F_2 - F_3 - F_4)],
[tau_1 - tau_2 + tau_3 - tau_4]]))

Now as we established above, the motor's torque and force are coupled via $\kappa$. Recall that

In [575]:
display(torques_from_forces)

Eq(Matrix([
[tau_1],
[tau_2],
[tau_3],
[tau_4]]), Matrix([
[F_1*kappa],
[F_2*kappa],
[F_3*kappa],
[F_4*kappa]]))

In [576]:
eq_tau_xyz_from_forces = \
    eq_tau_xyz.subs(tau_1, F1*kappa)\
              .subs(tau_2, F2*kappa)\
              .subs(tau_3, F3*kappa)\
              .subs(tau_4, F4*kappa).simplify()
display(eq_tau_xyz_from_forces)

Eq(Matrix([
[tau_x],
[tau_y],
[tau_z]]), Matrix([
[    l*(F_1 - F_2 - F_3 + F_4)],
[    l*(F_1 + F_2 - F_3 - F_4)],
[kappa*(F_1 - F_2 + F_3 - F_4)]]))

Right now, we have four degrees of freedom with only three controlling variables; however, we do know that

$$
F = F_1 + F_2 + F_3 + F_4.
$$

In [577]:
system = sym.Eq(
    sym.Matrix((F, tau_x, tau_y, tau_z)),
    sym.Matrix((
        total_thrust.rhs,
        eq_tau_xyz_from_forces.rhs[0],
        eq_tau_xyz_from_forces.rhs[1],
        eq_tau_xyz_from_forces.rhs[2]
    ))
)
display(system)

Eq(Matrix([
[    F],
[tau_x],
[tau_y],
[tau_z]]), Matrix([
[        F_1 + F_2 + F_3 + F_4],
[    l*(F_1 - F_2 - F_3 + F_4)],
[    l*(F_1 + F_2 - F_3 - F_4)],
[kappa*(F_1 - F_2 + F_3 - F_4)]]))

When we separate the variables, we get the matrix

In [578]:
result_vec = sym.Matrix((F, tau_x, tau_y, tau_z))
system_mtx = sym.Matrix(((1,  1,  1,  1), 
                         (l, -l, -l,  l),
                         (l,  l, -l, -l),
                         (kappa,-kappa,  kappa, -kappa)))

display(system_mtx)

Matrix([
[    1,      1,     1,      1],
[    l,     -l,    -l,      l],
[    l,      l,    -l,     -l],
[kappa, -kappa, kappa, -kappa]])

Checking the determinant tells us that the rows are independent (a condition that is required for a matrix to be invertible):

In [579]:
system_mtx.det()

-16*kappa*l**2

Let's quickly verify the system is indeed correct:

In [580]:
display(sym.Eq(result_vec, system_mtx * forces).simplify())

Eq(Matrix([
[    F],
[tau_x],
[tau_y],
[tau_z]]), Matrix([
[        F_1 + F_2 + F_3 + F_4],
[    l*(F_1 - F_2 - F_3 + F_4)],
[    l*(F_1 + F_2 - F_3 - F_4)],
[kappa*(F_1 - F_2 + F_3 - F_4)]]))

In [581]:
results = sym.solve(sym.Eq(result_vec, system_mtx * forces), forces, simplify=False)

for key in results:
    display(sym.Eq(key, results[key]))

Eq(F_1, F/4 + tau_x/(4*l) + tau_y/(4*l) + tau_z/(4*kappa))

Eq(F_2, F/4 - tau_x/(4*l) + tau_y/(4*l) - tau_z/(4*kappa))

Eq(F_3, F/4 - tau_x/(4*l) - tau_y/(4*l) + tau_z/(4*kappa))

Eq(F_4, F/4 + tau_x/(4*l) - tau_y/(4*l) - tau_z/(4*kappa))

In [582]:
system_mtx.inv()*result_vec

Matrix([
[F/4 + tau_x/(4*l) + tau_y/(4*l) + tau_z/(4*kappa)],
[F/4 - tau_x/(4*l) + tau_y/(4*l) - tau_z/(4*kappa)],
[F/4 - tau_x/(4*l) - tau_y/(4*l) + tau_z/(4*kappa)],
[F/4 + tau_x/(4*l) - tau_y/(4*l) - tau_z/(4*kappa)]])